In [27]:
X = []
Y = []

for line in open('train_data.txt').readlines():
    line = line.strip()
    a = line.index('[')
    b = line.index(']')
    grid = list(map(int, line[a+1: b].split(', ')))
    result = int(line[b+1:].strip())
    X.append(grid)
    Y.append(result)

In [28]:
import numpy as np
from sklearn.decomposition import PCA

X = np.array(X)
Y = np.array(Y)


# seeing -1 as 0 because there are only few cases like that
# qsvc doesn't support non-binary classification

Y[Y == -1] = 0
pca = PCA(n_components=7)
X = pca.fit_transform(X)  # the device only has 7 cores so we should use PCA to reduce the dimensions

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

features = MinMaxScaler(feature_range=(0, np.pi)).fit_transform(X)

train_features, test_features, train_labels, test_labels = train_test_split(
    X, Y, train_size=0.7, shuffle=True
)

In [11]:
# number of qubits is equal to the number of features
num_qubits = 2

# number of steps performed during the training procedure
tau = 100

# regularization parameter
C = 1000

In [23]:
from qiskit import *
IBMQ.providers()
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.utils import algorithm_globals
from qiskit.circuit.library import ZFeatureMap
from qiskit_machine_learning.algorithms import PegasosQSVC


from qiskit.visualization import plot_histogram

IBMQ.save_account('53fbe5f38c448c1c5d19a96fa47bd10ee765b1a9721453022212a2f5b3b29c0fb75a665962ef38dd5fde63cd26ddb2cd9d86fe78c9a37b947ff91e6bf911b0ea', overwrite=True)

IBMQ.load_account()

provider = IBMQ.get_provider(hub = 'ibm-q')

# devices = provider.backends()
# for _device in devices:
#     print(_device.name(), _device.configuration().n_qubits)
#     if _device.configuration().n_qubits >= 9:
#         device = _device
device = provider.get_backend('ibm_oslo')

ibmqfactory.load_account:WARNING:2023-01-29 04:34:30,349: Credentials are already in use. The existing account in the session will be replaced.


In [31]:
from qiskit_machine_learning.kernels import QuantumKernel

algorithm_globals.random_seed = 12345

feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)

pegasos_backend = QuantumInstance(backend = device)

qkernel = QuantumKernel(feature_map=feature_map, quantum_instance=pegasos_backend)

pegasos_qsvc = PegasosQSVC(quantum_kernel=qkernel, C=C, num_steps=tau)

pegasos_qsvc.fit(train_features, train_labels)


# testing
pegasos_score = pegasos_qsvc.score(test_features, test_labels)
print(f"PegasosQSVC classification test score: {pegasos_score}")

KeyboardInterrupt: 